<a href="https://colab.research.google.com/github/Bareeraq/sentiment-analysis-tool/blob/main/Sentiment_analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.version.cuda)  # Should output the installed CUDA version
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should show the number of GPUs available

12.1
True
1


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers

In [ ]:
!pip install huggingface_hub

In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


# **PREPROCESSING**

In [ ]:
from datasets import load_dataset

ds = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

print(ds)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

train_df.csv:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

val_df.csv:   0%|          | 0.00/601k [00:00<?, ?B/s]

test_df.csv:   0%|          | 0.00/586k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5205 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5206 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 31232
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5206
    })
})


In [ ]:
from datasets import DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load your dataset splits
train_dataset = ds["train"]  # Original train split
validation_dataset = ds["validation"]  # Original validation split
test_dataset = ds["test"]  # Original test split

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# **TOKENIZE THE DS**

In [ ]:
# from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    # Ensure 'text' is a string or a list of strings, and handle None values
    texts = examples["text"]
    if texts is None:  # Handle None values
        texts = [""]  # Replace None with empty string
    elif isinstance(texts, str):
        texts = [texts]  # Wrap single string in a list
    # Filter out any non-string elements in the list
    texts = [t for t in texts if isinstance(t, str)]

    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)

# Now proceed with mapping the tokenize_function:
train_dataset = train_dataset.map(tokenize_function, batched=True)
validation_dataset = validation_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batch_size=1000, drop_last_batch=True)

Map:   0%|          | 0/31232 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

Map:   0%|          | 0/5206 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset.column_names)
print(validation_dataset.column_names)
print(test_dataset.column_names)

['id', 'text', 'label', 'sentiment', 'input_ids', 'attention_mask']
['id', 'text', 'label', 'sentiment', 'input_ids', 'attention_mask']
['id', 'text', 'label', 'sentiment', 'input_ids', 'attention_mask']


In [ ]:
from transformers import DataCollatorWithPadding
# Prepare the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load the pretrained model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)  # Our dataset contains 3 classes

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **TRAINING THE MODEL**

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


## COMPUTE METRICS FUNCTION

In [ ]:
!pip install scikit-learn

In [ ]:
import sklearn
print(sklearn.__version__)

1.6.0


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from evaluate import load

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Handle single-class edge cases
    if len(set(labels)) == 1:  # Only one class in references
        return {"accuracy": 0.0, "f1": 0.0}  # Fallback values

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute F1 score manually without relying on `average`
    try:
        f1 = f1_score(labels, predictions, average=None).mean()  # Mean of F1 scores across all classes
    except Exception as e:
        print(f"F1 computation error: {e}")
        f1 = 0.0  # Fallback value

    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install wandb

In [ ]:
import wandb
print(wandb.__version__)

0.19.1


In [ ]:
# Define training arguments

repo_name = "Sentiment-analysis-tool"

training_args = TrainingArguments(
    run_name = "sentiment_analysis",    #specify a run name
    output_dir= repo_name,              # Directory for saving model checkpoints
    eval_strategy="epoch",              # Evaluate at the end of every epoch
    learning_rate=2e-5,                 # Learning rate
    per_device_train_batch_size=16,     # Batch size for training
    per_device_eval_batch_size=16,      # Batch size for evaluation
    num_train_epochs=3,                 # Number of training epochs
    weight_decay=0.01,                  # Weight decay for regularization
    logging_dir="./logs",               # Directory for logging
    logging_steps=10,                   # Log every 10 steps
    save_total_limit=2,                 # Keep only the last 2 checkpoints
    save_strategy="epoch",              # Save model checkpoints after every epoch
    push_to_hub=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,#FROM PREVIOUS CODE
)


<ipython-input-24-c3a30c352e90>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
wandb.init(project="sentiment-analysis-tool", name="sentiment-analysis") # Initialize wandb run
wandb.login()
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.551000,0.578788,0.760615,0.762635
2,0.547100,0.587074,0.764073,0.767081
3,0.302200,0.637914,0.760038,0.762810


No files have been modified since last commit. Skipping to prevent empty commit.


TrainOutput(global_step=5856, training_loss=0.5032300895761922, metrics={'train_runtime': 4703.5786, 'train_samples_per_second': 19.92, 'train_steps_per_second': 1.245, 'total_flos': 1.2411886728904704e+16, 'train_loss': 0.5032300895761922, 'epoch': 3.0})

In [ ]:
print(test_dataset.column_names)
print(test_dataset[0])  # Look at one example

['id', 'text', 'label', 'sentiment', 'input_ids', 'attention_mask']
{'id': 9235, 'text': 'getting cds ready for tour', 'label': 1, 'sentiment': 'neutral', 'input_ids': [[101, 2893, 14340, 3201, 2005, 2778, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
def flatten_nested(example):
    example["input_ids"] = example["input_ids"][0]
    example["attention_mask"] = example["attention_mask"][0]
    return example

test_dataset = test_dataset.map(flatten_nested)

Map:   0%|          | 0/5206 [00:00<?, ? examples/s]

In [ ]:
# Evaluate on the test set
results = trainer.evaluate(test_dataset)
print("Test results:", results)

Test results: {'eval_loss': 0.6321114897727966, 'eval_accuracy': 0.7668075297733384, 'eval_f1': 0.7697698116594229, 'eval_runtime': 81.6488, 'eval_samples_per_second': 63.761, 'eval_steps_per_second': 3.993, 'epoch': 3.0}
